In [45]:
%pip install xlsxwriter

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from math import sqrt
from collections import Counter

# Fungsi untuk menentukan keanggotaan Gaussian
def gaussian_membership(value, center, sigma):
    return np.exp(-0.5 * ((value - center) / sigma) ** 2)

# Fungsi untuk defuzzifikasi
def defuzzify(fuzzy_set, interval_labels, intervals):
    mapping = {label: (intervals[i], intervals[i + 1]) for i, label in enumerate(interval_labels)}
    return np.mean(mapping[fuzzy_set])

# Fungsi untuk memeriksa stabilitas prediksi
def stable_state_check(possible_states, last_values):
    if len(set(last_values[-3:])) == 1:
        return True
    return False

# Streamlit UI
st.title("Fuzzy Forecasting for Sales Data")

# Upload file
uploaded_file = st.file_uploader("Upload CSV file", type="csv")
if uploaded_file:
    df = pd.read_csv(uploaded_file)
    st.write("### Data Preview")
    st.write(df.head())

    # Resampling and preprocessing
    try:
        df.index = pd.to_datetime(df['Date'])
        df = df.drop(columns=['Date'])
    except Exception as e:
        st.error("Error processing the 'Date' column: " + str(e))

    df_resampled = df.resample('M').sum()
    produk_columns = df_resampled.columns.tolist()

    metrics_list = []  # Menyimpan hasil metrik sebagai list
    all_forecasts = {}  # Menyimpan semua prediksi untuk perbandingan antar produk

    # Loop untuk setiap produk
    for produk in produk_columns:
        U_min = df_resampled[produk].min()
        U_max = df_resampled[produk].max()
        U = np.linspace(U_min, U_max, num=100)

        quantiles = df_resampled[produk].quantile(np.linspace(0, 1, num=10)).values
        interval_labels = [f"Interval {i+1}" for i in range(len(quantiles) - 1)]
        intervals = quantiles

        def fuzzify(value):
            memberships = [
                gaussian_membership(value, (intervals[i] + intervals[i + 1]) / 2, (intervals[i + 1] - intervals[i]) / 4)
                for i in range(len(interval_labels))
            ]
            return interval_labels[np.argmax(memberships)]

        df_resampled[f"Fuzzy State {produk}"] = df_resampled[produk].apply(fuzzify)

        relations = []
        for i in range(len(df_resampled) - 1):
            current_state = df_resampled[f"Fuzzy State {produk}"].iloc[i]
            next_state = df_resampled[f"Fuzzy State {produk}"].iloc[i + 1]
            relations.append((current_state, next_state))

        rules = {}
        for (current, next_) in relations:
            if current in rules:
                rules[current].append(next_)
            else:
                rules[current] = [next_]

        predicted_values = []
        current_state = df_resampled[f"Fuzzy State {produk}"].iloc[-1]
        last_predictions = []

        for i in range(12):
            possible_states = rules.get(current_state, [])

            if possible_states:
                predicted_state = Counter(possible_states).most_common(1)[0][0]
                predicted_value = defuzzify(predicted_state, interval_labels, intervals)
            else:
                predicted_value = defuzzify(current_state, interval_labels, intervals)

            last_predictions.append(predicted_value)
            predicted_values.append(predicted_value)

            if len(last_predictions) >= 3 and stable_state_check(possible_states, last_predictions):
                while len(predicted_values) < 12:
                    predicted_values.append(predicted_value)
                break

            current_state = predicted_state

        predicted_dates = pd.date_range(start=df_resampled.index[-1] + pd.DateOffset(months=1), periods=12, freq='M')
        forecast_df = pd.DataFrame({
            f"Actual {produk}": df_resampled[produk],
            f"Predicted {produk}": pd.Series(predicted_values, index=predicted_dates)
        })

        st.write(f"### Forecast for {produk}")
        st.line_chart(forecast_df)

        actual_sales = df_resampled[produk].iloc[-12:]
        mae = mean_absolute_error(actual_sales, predicted_values)
        mape = np.mean(np.abs((actual_sales - predicted_values) / actual_sales)) * 100
        rmse = sqrt(mean_squared_error(actual_sales, predicted_values))

        metrics_list.append({"Product": produk, "MAE": mae, "MAPE": mape, "RMSE": rmse})

    metrics_df = pd.DataFrame(metrics_list)
    st.write("### Metrics Summary")
    st.dataframe(metrics_df)


ModuleNotFoundError: No module named 'sklearn'